In [1]:
from keras.preprocessing.image import img_to_array
import cv2
from keras.models import load_model
import numpy as np

# load face detection haar cascade
cascade_path = 'haarcascade_frontalface_default.xml'
face_detection = cv2.CascadeClassifier(cascade_path)

# load our pretrained model
emotion_model_path = 'logs/model_mini_XCEPTION.29-0.60.hdf5'
emotion_classifier = load_model(emotion_model_path, compile=False)

# decoded list of emotions
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised", "neutral"]

Using TensorFlow backend.


In [ ]:
cv2.namedWindow('emotion_recognition')
camera = cv2.VideoCapture(0)

while True: 
    # read one frame and preprocess it
    frame = camera.read()[1]
    (h, w, d) = frame.shape
    r = 400 / w
    dim = (400, int(h * r))
    frame = cv2.resize(frame, dim)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   
    
    # detect face
    faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    frameClone = frame.copy()
    
    for (x, y, w, h) in faces: 
        cv2.rectangle(frameClone, (x, y), ((x + w), (y + h)), (0, 0, 255), 2)  
        if len(faces)>0:
            # calculate ROIs squares
            S = []
            for face in faces:
                (fX, fY, fW, fH) = face
                S.append(fW*fH)
            # take face with largest square
            face_index = S.index(max(S))
            face = faces[face_index]

            (fX, fY, fW, fH) = face
            
            # define ROI
            roi = gray[fY:fY + fH, fX:fX + fW]
            roi = cv2.resize(roi, (48, 48))
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            
            # make prediction
            preds = emotion_classifier.predict(roi)[0]
            emotion_probability = np.max(preds)
            label = EMOTIONS[preds.argmax()]
            
            emotions_spectrum = enumerate(zip(EMOTIONS, preds))
            for (i, (emotion, probability)) in emotions_spectrum:
                text = emotion + ": " + str("{0:.1f}".format(probability*100)) + '%'
                w = int(probability * 120)
                cv2.rectangle(frameClone, (2, (i * 30) + 5),(120, (i * 30) + 30), (220, 220, 220), -1)
                cv2.rectangle(frameClone, (2, (i * 30) + 5),(w, (i * 30) + 30), (0, 185, 0), -1)
                cv2.putText(frameClone, text, (10, (i * 30) + 21),cv2.FONT_HERSHEY_SIMPLEX, 0.45,(0, 0, 255), 1)
                  
            cv2.putText(frameClone, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)
        
    cv2.imshow('Emotion Recognition', frameClone)
    if cv2.waitKey(1) & 0xff == ord('q'):                
        break
    
camera.release()                                  
cv2.destroyAllWindows() 